In [21]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import urllib
import re
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pathlib import Path

In [22]:
def get_data_url(target_month, place):
    year = target_month.strftime("%Y")
    month = target_month.strftime("%m")
    base = 'https://keirin.kdreams.jp'
    baseAdd1 = "/" + place + "/schedule/" + year + "/" + month
    url = base + baseAdd1
    return urllib.parse.quote_plus(url, "/:?=&")

In [23]:
def month_span(start_month, end_month):
    """start_date、end_dateの期間に含まれる日毎のdatetimeオブジェクトを返すジェネレータ
    """
    i = 0
    while 1 :
        target_month = start_month + relativedelta(months=i)
        if target_month > end_month: break
        yield target_month
        i += 1
        

In [35]:
start_month = datetime.strptime('201708', '%Y%m')
end_month = datetime.strptime('201708', '%Y%m')

today = datetime.today()
date_prefix = today.strftime('%Y%m%d-%H%M%S')
file_origin = 'crawl_' + date_prefix + '_Crawler.csv'

my_file = Path(file_origin)
isFirst = True

if my_file.is_file():
    mainDf = pd.read_csv(file_origin, index_col=0, header=0)
    isFirst = False
else:
    mainDf = None
    

In [36]:
places = ['hakodate']

sleep_time = 5

In [52]:
for target_month in month_span(start_month, end_month):
    for place in places:
        target_url = get_data_url(target_month, place)
        print(target_url)
        headers = {'User-Agent': 'Mozilla/5.0'}
        time.sleep(sleep_time)
        response = requests.get(target_url, headers=headers)# <Response [200]>
        soup = BeautifulSoup(response.text, 'html.parser')
        dokantos = soup.find_all("p", class_="dokanto")
        for dokanto in dokantos:
            raceresult_link = dokanto.a.get("href")
            print(raceresult_link)
            time.sleep(sleep_time)
            response = requests.get(raceresult_link, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            racedetails_bs = soup.find_all("p", class_="race")
            for racedetail_bs in racedetails_bs:
                racedetail_link = racedetail_bs.a.get('href')
                print(racedetail_link)
                time.sleep(sleep_time)
                response = requests.get(racedetail_link, headers=headers)
                soup = BeautifulSoup(response.text, 'html.parser')
                print("RACE RACORD")
                racecard_bs = soup.find("table", class_="racecard_table")
                #print(racecard)
                riders_bs = racecard_bs.find_all("tr", class_=re.compile("^n"))
                for rider_bs in riders_bs:
                    rows = []
                    for td in rider_bs.find_all("td"):
                        row = td.text
                        row = ''.join(row.split())
                        rows.append(row)
                    if len(rows) < 23:
                        rows.insert(3, bracket)
                    
                    names = rider_bs.find("td", class_="rider bdr_r").get_text().strip().splitlines()
                    name = names[0]
                    homes = ''.join(names[1].split()).split("/")
                    prefecture = homes[0]
                    age = homes[1]
                    period = homes[2]
                    print(name, home, age, period)
                    
                    rows.pop(5)
                    rows.insert(5, name)
                    rows.insert(6, prefecture)
                    rows.insert(7, age)
                    rows.insert(8, period)
                    
                    #print(rows)
                    
                    rows[0] = rows[0].replace('◎', '1').replace('○', '2').replace('△', '3').replace('▲', '4').replace('×', '5').replace('注', '6').replace('', '10')
                    bracket = rows[3]
                    rows[6] = rows[6].replace('北海道', '1').replace('青森', '2').replace('岩手', '3').replace('宮城', '4').replace('秋田', '5').replace('山形', '6').replace('福島', '7').replace('茨城', '8').replace('栃木', '9').replace('群馬', '10').replace('埼玉', '11').replace('千葉', '12').replace('東京', '13').replace('神奈川', '14').replace('新潟', '15').replace('富山', '16').replace('石川', '17').replace('福井', '18').replace('山梨', '19').replace('長野', '20').replace('岐阜', '21').replace('静岡', '22').replace('愛知', '23').replace('三重', '24').replace('滋賀', '25').replace('京都', '26').replace('大阪', '27').replace('兵庫', '28').replace('奈良', '29').replace('和歌山', '30').replace('鳥取', '31').replace('島根', '32').replace('岡山', '33').replace('広島', '34').replace('山口', '35').replace('徳島', '36').replace('香川', '37').replace('愛媛', '38').replace('高知', '39').replace('福岡', '40').replace('佐賀', '41').replace('長崎', '42').replace('熊本', '43').replace('大分', '44').replace('宮崎', '45').replace('鹿児島', '46').replace('沖縄', '47')
                    rows[9] = rows[9].replace('S1', '1').replace('S2', '2').replace('A1', '3').replace('A2', '4').replace('A3', '5')
                    rows[10] = rows[10].replace('両', '1').replace('逃', '2').replace('追', '3')
                    
                    print(rows)
                    
                #print("RACE RESULT")
                #print(soup.find("table", class_="result_table"))
    


https://keirin.kdreams.jp/hakodate/schedule/2017/08
https://keirin.kdreams.jp/hakodate/raceresult/11201707310100/
https://keirin.kdreams.jp/hakodate/racedetail/1120170731010001/?pageType=KS_RACE_CARD_PAGE_TYPE_SHOW_RESULT
RACE RACORD
伊藤 司 福島 39 83
['◎', '', '17', '1', '1', '伊藤 司', '福島', '39', '83', 'A3', '両', '3.92', '78.08', '1', '2', '1', '2', '2', '0', '2', '3', '2', '16', '8.6', '21.7', '30.4']
['10110', '', '17', '1', '1', '伊藤 司', '7', '39', '83', '5', '1', '3.92', '78.08', '1', '2', '1', '2', '2', '0', '2', '3', '2', '16', '8.6', '21.7', '30.4']
蕗澤 鴻太郎 福島 22 111
['×', '', '24', '2', '2', '蕗澤 鴻太郎', '群馬', '22', '111', 'A3', '逃', '3.71', '68.33', '0', '2', '2', '0', '0', '0', '1', '1', '0', '1', '33.3', '66.6', '66.6']
['10510', '', '24', '2', '2', '蕗澤 鴻太郎', '10', '22', '111', '5', '2', '3.71', '68.33', '0', '2', '2', '0', '0', '0', '1', '1', '0', '1', '33.3', '66.6', '66.6']
堀 検正 福島 38 81
['▲', '', '24', '3', '3', '堀 検正', '熊本', '38', '81', 'A3', '追', '3.79', '68.61', '1', '0', '0',

RACE RACORD
野口 裕史 福島 34 111
['◎', '', '12', '1', '1', '野口 裕史', '千葉', '34', '111', 'A3', '逃', '3.85', '79.33', '2', '6', '5', '1', '0', '0', '5', '1', '0', '0', '83.3', '100.0', '100.0']
['10110', '', '12', '1', '1', '野口 裕史', '12', '34', '111', '5', '2', '3.85', '79.33', '2', '6', '5', '1', '0', '0', '5', '1', '0', '0', '83.3', '100.0', '100.0']
百瀬 匡 福島 39 83
['▲', '', '25', '2', '2', '百瀬 匡', '長野', '39', '83', 'A3', '両', '3.77', '67.61', '3', '0', '0', '2', '3', '1', '5', '1', '0', '15', '23.8', '28.5', '28.5']
['10410', '', '25', '2', '2', '百瀬 匡', '20', '39', '83', '5', '1', '3.77', '67.61', '3', '0', '0', '2', '3', '1', '5', '1', '0', '15', '23.8', '28.5', '28.5']
丹波 福道 福島 45 74
['△', '', '23', '3', '3', '丹波 福道', '岡山', '45', '74', 'A3', '追', '3.83', '69.50', '0', '0', '0', '0', '3', '2', '1', '4', '4', '13', '4.5', '22.7', '40.9']
['10310', '', '23', '3', '3', '丹波 福道', '33', '45', '74', '5', '3', '3.83', '69.50', '0', '0', '0', '0', '3', '2', '1', '4', '4', '13', '4.5', '22.7', '40.9'

RACE RACORD
廣川 泰昭 福島 34 90
['○', '', '17', '1', '1', '廣川 泰昭', '愛媛', '34', '90', 'A2', '追', '3.92', '89.08', '1', '1', '0', '1', '1', '4', '2', '4', '4', '14', '8.3', '25.0', '41.6']
['10210', '', '17', '1', '1', '廣川 泰昭', '38', '34', '90', '4', '3', '3.92', '89.08', '1', '1', '0', '1', '1', '4', '2', '4', '4', '14', '8.3', '25.0', '41.6']
小原 伸哉 福島 29 97
['▲', '', '20', '2', '2', '小原 伸哉', '北海道', '29', '97', 'A2', '追', '3.86', '84.50', '3', '0', '0', '0', '2', '1', '1', '2', '1', '17', '4.7', '14.2', '19.0']
['10410', '', '20', '2', '2', '小原 伸哉', '1', '29', '97', '4', '3', '3.86', '84.50', '3', '0', '0', '0', '2', '1', '1', '2', '1', '17', '4.7', '14.2', '19.0']
長尾 博幸 福島 37 87
['△', '', '16', '3', '3', '長尾 博幸', '大阪', '37', '87', 'A1', '両', '3.85', '85.58', '1', '2', '0', '4', '1', '0', '3', '2', '2', '10', '17.6', '29.4', '41.1']
['10310', '', '16', '3', '3', '長尾 博幸', '27', '37', '87', '3', '1', '3.85', '85.58', '1', '2', '0', '4', '1', '0', '3', '2', '2', '10', '17.6', '29.4', '41.1']
後藤

https://keirin.kdreams.jp/hakodate/racedetail/1120170819010001/?pageType=KS_RACE_CARD_PAGE_TYPE_SHOW_RESULT
RACE RACORD
廣瀬 裕道 福島 43 71
['◎', '', '16', '1', '1', '廣瀬 裕道', '茨城', '43', '71', 'A1', '追', '3.86', '88.00', '0', '0', '0', '0', '2', '4', '2', '4', '4', '14', '8.3', '25.0', '41.6']
['10110', '', '16', '1', '1', '廣瀬 裕道', '8', '43', '71', '3', '3', '3.86', '88.00', '0', '0', '0', '0', '2', '4', '2', '4', '4', '14', '8.3', '25.0', '41.6']
石井 雅典 福島 46 68
['×', '', '19', '2', '2', '石井 雅典', '千葉', '46', '68', 'A2', '追', '3.77', '85.38', '0', '0', '0', '0', '1', '4', '1', '4', '2', '15', '4.5', '22.7', '31.8']
['10510', '', '19', '2', '2', '石井 雅典', '12', '46', '68', '4', '3', '3.77', '85.38', '0', '0', '0', '0', '1', '4', '1', '4', '2', '15', '4.5', '22.7', '31.8']
高桑 由昇 福島 39 81
['', '', '19', '3', '3', '高桑 由昇', '福島', '39', '81', 'A1', '追', '3.92', '82.92', '0', '0', '0', '0', '2', '3', '2', '3', '0', '22', '7.4', '18.5', '18.5']
['10', '', '19', '3', '3', '高桑 由昇', '7', '39', '81', '3'

RACE RACORD
猪狩 祐樹 福島 30 99
['◎', '', '12', '1', '1', '猪狩 祐樹', '福島', '30', '99', 'A1', '逃', '3.93', '89.33', '0', '12', '4', '4', '1', '0', '7', '2', '1', '11', '33.3', '42.8', '47.6']
['10110', '', '12', '1', '1', '猪狩 祐樹', '7', '30', '99', '3', '2', '3.93', '89.33', '0', '12', '4', '4', '1', '0', '7', '2', '1', '11', '33.3', '42.8', '47.6']
沼田 淳一郎 福島 39 85
['×', '', '17', '2', '2', '沼田 淳一郎', '神奈川', '39', '85', 'A1', '追', '3.93', '87.06', '0', '0', '0', '0', '2', '2', '2', '2', '3', '8', '13.3', '26.6', '46.6']
['10510', '', '17', '2', '2', '沼田 淳一郎', '14', '39', '85', '3', '3', '3.93', '87.06', '0', '0', '0', '0', '2', '2', '2', '2', '3', '8', '13.3', '26.6', '46.6']
長岡 豪 福島 34 91
['▲', '', '17', '3', '3', '長岡 豪', '群馬', '34', '91', 'A1', '追', '3.86', '85.37', '0', '0', '0', '1', '5', '2', '1', '7', '2', '20', '3.3', '26.6', '33.3']
['10410', '', '17', '3', '3', '長岡 豪', '10', '34', '91', '3', '3', '3.86', '85.37', '0', '0', '0', '1', '5', '2', '1', '7', '2', '20', '3.3', '26.6', '33.3']


KeyboardInterrupt: 

In [121]:
        
            target_url = get_data_url(target_month.strftime('%Y%m'), pid, rno)
            date = target_date.strftime('%Y%m%d')
            headers = {'User-Agent': 'Mozilla/5.0'}
            time.sleep(sleep_time)
            response = requests.get(target_url, headers=headers)# <Response [200]>
            soup = BeautifulSoup(response.text, 'html.parser')
            content1 = soup.find_all("h1", class_="h_content1")
            content2 = soup.find_all("div", class_="blocks")
            content3 = soup.find_all("table", id="detail_program")
            for content3_part in content3:
                tds1 = [td.find_all("p") for td in content3_part.find_all("td", class_="border_all name_kanji")]
                ID = [row[0].string for row in tds1]
                name = [row[1].string for row in tds1]
                prefecture = [row[2].string.split("/") for row in tds1]
                age = [row[3].string.split("/") for row in tds1]
                ages = [row[0].replace('歳', '') for row in age]
                term = [row[1].replace('期', '') for row in age]
                prefectures = [row[0].replace('北海道', '1').replace('青森', '2').replace('岩手', '3').replace('宮城', '4').replace('秋田', '5').replace('山形', '6').replace('福島', '7').replace('茨城', '8').replace('栃木', '9').replace('群馬', '10').replace('埼玉', '11').replace('千葉', '12').replace('東京', '13').replace('神奈川', '14').replace('新潟', '15').replace('富山', '16').replace('石川', '17').replace('福井', '18').replace('山梨', '19').replace('長野', '20').replace('岐阜', '21').replace('静岡', '22').replace('愛知', '23').replace('三重', '24').replace('滋賀', '25').replace('京都', '26').replace('大阪', '27').replace('兵庫', '28').replace('奈良', '29').replace('和歌山', '30').replace('鳥取', '31').replace('島根', '32').replace('岡山', '33').replace('広島', '34').replace('山口', '35').replace('徳島', '36').replace('香川', '37').replace('愛媛', '38').replace('高知', '39').replace('福岡', '40').replace('佐賀', '41').replace('長崎', '42').replace('熊本', '43').replace('大分', '44').replace('宮崎', '45').replace('鹿児島', '46').replace('沖縄', '47') for row in prefecture]
                rank = [row[1].replace('A1', '1').replace('A2', '2').replace('B1', '3').replace('B2', '4') for row in prefecture]
                tds2 = [td.text for td in content3_part.find_all("td", class_="border_all average")[2:][::5]]
                morter_num = [td[:-5] for td in tds2]
                for content2_part in content2:
                    race_number = [p.text.replace('【', '').replace('】', '').replace('予選', '').replace('R', '') for p in content2_part.find_all("p", class_="left")]
                    for num in race_number:
                        race_number = [num for i in range(6)]
                        race_number = [num[:3] for num in race_number]
                for content1_part in content1:
                    race_place = [n[:3] for n in content1_part]
                    race_place = race_place * 6
                raceDict = pd.DataFrame({"date": date,                                         
                                        "ID": ID,
                                        "name": name,
                                        "age": ages,
                                        "term": term,
                                        "prefecture": prefectures,
                                        "rank": rank,
                                        "morter_num": morter_num,
                                        "race_number": race_number,
                                        "race_place": race_place
                                         })
                print(raceDict)
                # 取得した後、追加していく
                if mainDf is None:
                    raceDict.to_csv(file_origin)
                    mainDf = raceDict
                else:
                    raceDict.to_csv(file_origin, mode='a', header=False)
                    mainDf = mainDf.append(raceDict)
                                    
    print("Completed.")

IndentationError: unexpected indent (<ipython-input-121-b3413e40a238>, line 2)